# Readme


* This notebook can be either run with "Cell" - "Run All", or with "Cell" - "Run Cells" (from the above menu)
  * With the first option ("Run All"), the entire script will be automatically executed till the end.
  * With the second option ("Run Cells"), only the code of a single cell will be executed. Clicking THIS it is possibile to procede, cell by cell, till the end of the code.
  
 ![skipforward](https://raw.githubusercontent.com/antonelepfl/testvue/master/skip_next.png "size=10")


* Each time a notebook is executed, a kernel status symbol is displayed in the top right corner of the notebook
  * When there is a circle bullet •, it means that the kernel is running and the user have to be sure to not interfere with the code execution
  * When there is an empty circle bullet ○, it means that the kernel is idle and the user can interfere with the notebook.
  
See Guidebook at https://collab.humanbrainproject.eu/#/collab/1655/nav/18580

## Hint for this notebook

Since the simulations are computational intensive, this optimization can be run on the Collab platform with a maximum of 6 individuals for only 6 generations.

### Preparation
#### Define the Simulation parameters

Define the number of individuals and the number of generations.

In [2]:
NUMBER_INDIVIDUALS=6 # Number of individuals in offspring
NUMBER_GENERATIONS=6 #Maximum number of generations

In [ ]:
#installation of BluePyOpt
!pip2 install pip wget bluepyopt >/dev/null

This custom class define the parameters and section to create a custom built axon.
It is used to define the temperature, Vinit and the tables used in the MOD files too.

In [ ]:
#import of BluePyOpt
import bluepyopt as bpop
import bluepyopt.ephys as ephys	
import matplotlib.pyplot as plt
import os

class GrCMorphologymulti(ephys.morphologies.NrnFileMorphology):
  def __init__(self, morphology_path, do_replace_axon):

	super(GrCMorphologymulti, self).__init__(morphology_path, do_replace_axon)
	
  @staticmethod
  def replace_axon(sim=None, icell=None):
        """Replace axon"""

        # Define origin of distance function
        sim.neuron.h.distance(sec=icell.soma[0])

        for section in icell.axonal:
            sim.neuron.h.delete_section(sec=section)

        sim.neuron.h.execute('create axon[2]', icell)

#Hillock
        icell.axon[0].nseg = 1
        icell.axon[0].L = 2.5
        icell.axon[0].diam = 1.5
        icell.axon[0].Ra = 100
 
#axon 
        icell.axon[1].nseg = 3
        icell.axon[1].L = 70
        icell.axon[1].diam = 0.3
        icell.axon[1].Ra = 100


#Connections of the axon to the hillock and to the soma      
        icell.axon[0].connect(icell.soma[0], 1.0, 0.0)
        icell.axon[1].connect(icell.axon[0], 1.0, 0.0)
        
        icell.Hilock.append(sec = icell.axon[0])
        icell.AIS.append(sec = icell.axon[1]) 
    
#Temperature and v_init
        sim.neuron.h.celsius = 37
        sim.neuron.h.v_init = -70
        
#Ionic channels tables
        sim.neuron.h.usetable_GRC_KA = 0
        sim.neuron.h.usetable_GRC_KIR  = 0
        sim.neuron.h.usetable_GRC_KM = 0
        sim.neuron.h.usetable_GRC_KCA = 0
        sim.neuron.h.usetable_GRC_CA = 0
        sim.neuron.h.usetable_GRC_KV = 0

In [ ]:
! pip install --upgrade "hbp-service-client==1.0.0" >/dev/null

In [ ]:
#access to the collaboratory storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()

To run the optimization, it is necessary to download specific files, from the collab storage space, to the place where the python notebook will run. 
The files downloaded are: the neuron morphology (asc - neurolucida format) and the ionic channels (mod files).

In [ ]:
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GrC2017.asc', 'GrC2017.asc');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_CA.mod', 'GRC_CA.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_CALC.mod', 'GRC_CALC.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_KA.mod', 'GRC_KA.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_KCA.mod', 'GRC_KCA.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_KIR.mod', 'GRC_KIR.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_KM.mod', 'GRC_KM.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_KV.mod', 'GRC_KV.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_LKG1.mod', 'GRC_LKG1.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_LKG2.mod', 'GRC_LKG2.mod');
clients.storage.download_file('/BSP/OnlineUseCases/SingleCellModeling/Cerebellum_Opt/GrC_multi/GRC_NA.mod', 'GRC_NA.mod');

This command will compile the MOD files

In [ ]:
!nrnivmodl >/dev/null 2>/dev/null

First step in the definition of the BluePyOpt Granular cell template.
- The morphology, which can be a neurolucida file or an swc file.
- The locations for the ionic channels.
The basic sectionlists are somatic, axonal, apical and basal. 
It is possibile to define custom sections and sectionslists

In [ ]:
#Morphology location
morph = GrCMorphologymulti('GrC2017.asc', do_replace_axon = True)

#Locations
dend_loc = ephys.locations.NrnSeclistLocation('dend', seclist_name='dend')

somatic_loc = ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic')

axon0 = ephys.locations.NrnSeclistLocation('Hilock', seclist_name='Hilock')
axon1 = ephys.locations.NrnSeclistLocation('AIS', seclist_name='AIS')

This cell contains all the information to place the passive properties, with specific parameters, on the morphological locations previously defined. For example: the membrane capacitance on all sections, will have a name cm_all, a param name, taken from the mod file, the value, the location and the fact that is a fixed parameter (frozen = True).

In [ ]:
#DENDRITES.

#Passive properties 
cm_all = ephys.parameters.NrnSectionParameter(
        name='cm_all', 
        param_name='cm',
        value=1,
        locations=[dend_loc,somatic_loc, axon0, axon1],
        frozen=True)

ena = ephys.parameters.NrnSectionParameter(
        name='ena_all', 
        param_name='ena',
        value=87.39,
        locations=[axon0, axon1],
        frozen=True)

ek = ephys.parameters.NrnSectionParameter(
        name='ek_all', 
        param_name='ek',
        value=-84.69,
        locations=[dend_loc, somatic_loc, axon0, axon1],
        frozen=True)

eleak = ephys.parameters.NrnSectionParameter(
        name='eleak_all', 
        param_name='el_GRC_LKG1',
        value = -16.5,
        locations=[dend_loc, somatic_loc, axon0, axon1],
        frozen=True)

eca = ephys.parameters.NrnSectionParameter(
        name='eca_all', 
        param_name='eca',
        value = 129.33 ,
        locations=[dend_loc, somatic_loc],
        frozen=True)

Ra = ephys.parameters.NrnSectionParameter(
        name='ra_all', 
        param_name='Ra',
        value = 100,
        locations=[dend_loc, somatic_loc, axon0, axon1],
        frozen=True)

The ionic mechanism are loaded and inserted in the respective sections. The name must be unique instead the prefix is the same as the prefix contained in the MOD files.

In [ ]:
#CHANNELS

#Dendrites
GRC_KCA_dend = ephys.mechanisms.NrnMODMechanism(
        name='GRC_KCA_dend',
        prefix='GRC_KCA',
        locations=[dend_loc])

GRC_CA_dend = ephys.mechanisms.NrnMODMechanism(
        name='GRC_CA_dend',
        prefix='GRC_CA',
        locations=[dend_loc])

GrC_Leak_gaba = ephys.mechanisms.NrnMODMechanism(
        name='GRC_LKG2_dend',
        prefix='GRC_LKG2',
        locations=[dend_loc])

#Soma
GRC_KA_soma = ephys.mechanisms.NrnMODMechanism(
        name='GRC_KA',
        prefix='GRC_KA',
        locations=[somatic_loc])

GRC_KIR_soma = ephys.mechanisms.NrnMODMechanism(
        name='GRC_KIR',
        prefix='GRC_KIR',
        locations=[somatic_loc])

GRC_KM_soma = ephys.mechanisms.NrnMODMechanism(
        name='GRC_KM',
        prefix='GRC_KM',
        locations=[somatic_loc])

#Mixed soma, dendrites and axon.

GrC_Leak = ephys.mechanisms.NrnMODMechanism(
        name='Leak_all',
        prefix='GRC_LKG1',
        locations=[dend_loc, somatic_loc, axon0, axon1])

GRC_NA = ephys.mechanisms.NrnMODMechanism(
        name='GRC_NA',
        prefix='GRC_NA',
        locations=[axon0, axon1])

GRC_KV = ephys.mechanisms.NrnMODMechanism(
        name='GRC_KV',
        prefix='GRC_KV',
        locations=[axon0, axon1])

GRC_Calc = ephys.mechanisms.NrnMODMechanism(
        name='GRC_CALC',
        prefix='GRC_CALC',
        locations=[dend_loc, somatic_loc])

This is the section in which, each single ionic channel, receives its conductance range. The name is to be unique, the param_name is, again, taken from each MOD file. The bounds are the ranges of conductances and, since are a range, are not frozen.

In [ ]:
#CHANNELS PARAM

#Dendrites
GrC_Leak_param_dend = ephys.parameters.NrnSectionParameter(                                    
        name='gl_GRC_LKG1_dend',
        param_name='gl_GRC_LKG1',
        locations=[dend_loc],
        bounds=[1.6908225e-06, 0.0002479873],
        frozen=False)

GrC_Leak_gaba_param_dend = ephys.parameters.NrnSectionParameter(                                    
        name='ggaba_GRC_LKG2_dend',
        param_name='ggaba_GRC_LKG2',
        locations=[dend_loc],
        bounds=[3.57216e-06, 0.0005239168],
        frozen=False)

GrC_GRC_KCA_param_dend = ephys.parameters.NrnSectionParameter(
        name='gkbar_GRC_KCA_dend',
        param_name='gkbar_GRC_KCA',
        bounds=[0.000285, 0.0476],
        locations=[dend_loc],
        frozen=False)

GrC_GRC_CA_param_dend = ephys.parameters.NrnSectionParameter(
        name='gcabar_GRC_CA_dend',
        param_name='gcabar_GRC_CA',
        bounds=[0.000438, 0.143],
        locations=[dend_loc],
        frozen=False)

GrC_Calc_param_dend = ephys.parameters.NrnSectionParameter(
        name='beta_GRC_CALC_dend',
        param_name='beta_GRC_CALC',
        bounds=[0.6, 0.61],
        locations=[dend_loc],
        frozen=False)

GrC_cai0_Calc_param_dend = ephys.parameters.NrnSectionParameter(
        name='cai0_GRC_CALC_dend',
        param_name='cai0_GRC_CALC',
        bounds=[0.002, 0.0025],
        locations=[dend_loc],
        frozen=False)

#Soma
GRC_KIR_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_GRC_KIR_soma',
        param_name='gkbar_GRC_KIR',
        locations=[somatic_loc],
        bounds=[0.000191, 0.0318],
        frozen=False)

GRC_KA_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_GRC_KA_soma',
        param_name='gkbar_GRC_KA',
        locations=[somatic_loc],
        bounds=[0.0004, 0.1],
        frozen=False) 

GrC_Leak_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gl_GRC_LKG1_soma',
        param_name='gl_GRC_LKG1',
        locations=[somatic_loc],
        bounds=[8.04e-06, 0.00134],
        frozen=False)

GRC_KM_param_soma = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_GRC_KM_soma',
        param_name='gkbar_GRC_KM',
        locations=[somatic_loc],
        bounds=[1.875e-05, 0.003125],
        frozen=False) 

GrC_Calc_param_soma = ephys.parameters.NrnSectionParameter(
        name='beta_GRC_CALC_soma',
        param_name='beta_GRC_CALC',
        bounds=[1.49, 1.5],
        locations=[somatic_loc],
        frozen=False)

GrC_cai0_Calc_param_soma = ephys.parameters.NrnSectionParameter(
        name='cai0_GRC_CALC_soma',
        param_name='cai0_GRC_CALC',
        bounds=[0.0001, 0.00013],
        locations=[somatic_loc],
        frozen=False)

#Axon
#Hilock

GRC_NA_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GRC_NA_axon0',
        param_name='gnabar_GRC_NA',
        locations=[axon0],
        bounds=[0.01791, 3.886],
        frozen=False)  

GRC_KV_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_GRC_KV_axon0',
        param_name='gkbar_GRC_KV',
        locations=[axon0],
        bounds=[0.00267, 0.445],
        frozen=False) 

GrC_Leak_param_axon0 = ephys.parameters.NrnSectionParameter(                                    
        name='gl_GRC_LKG1_axon0',
        param_name='gl_GRC_LKG1',
        locations=[axon0],
        bounds=[7.214e-06, 0.0025],
        frozen=False)

#axon
GRC_NA_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gnabar_GRC_NA_axon1',
        param_name='gnabar_GRC_NA',
        locations=[axon1],
        bounds=[0.000174, 0.029],
        frozen=False)  

GRC_KV_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gkbar_GRC_KV_axon1',
        param_name='gkbar_GRC_KV',
        locations=[axon1],
        bounds=[0.00033, 0.0559],
        frozen=False) 

GrC_Leak_param_axon1 = ephys.parameters.NrnSectionParameter(                                    
        name='gl_GRC_LKG1_axon1',
        param_name='gl_GRC_LKG1',
        locations=[axon1],
        bounds=[6.4319e-07, 0.0001071],
        frozen=False)

Creating the template

To create the cell template, we pass all the objects, previously prepared, to the constructor of the template.
The section lists for the axon, are declared here and used in the morphology class.

In [ ]:
GrCmulti = ephys.models.CellModel(
        name='SMGrCmulti',
        morph=morph,
        mechs=[GRC_CA_dend,
               GRC_KCA_dend,
               GrC_Leak,
               GrC_Leak_gaba,
               GRC_Calc,
               GRC_KIR_soma, 
               GRC_KA_soma, 
               GRC_KM_soma,
               GRC_NA,
               GRC_KV],
        
        params=[cm_all,
            ena,
            ek,
            eleak,
            eca,
            Ra,
            GrC_GRC_CA_param_dend, 
            GrC_GRC_KCA_param_dend, 
            GrC_Leak_param_dend,
            GrC_Leak_gaba_param_dend,
            GrC_Calc_param_dend, 
            GrC_cai0_Calc_param_dend,
            GRC_KIR_param_soma,
            GRC_KA_param_soma,
            GrC_Leak_param_soma,
            GRC_KM_param_soma,
            GrC_Calc_param_soma,
            GrC_cai0_Calc_param_soma,
            GRC_NA_param_axon0,
            GRC_KV_param_axon0,
            GrC_Leak_param_axon0,
            GRC_NA_param_axon1,
            GRC_KV_param_axon1,
            GrC_Leak_param_axon1])  

#custom sectionlists for the axon
GrCmulti.seclist_names.append('Hilock')
GrCmulti.seclist_names.append('AIS')

print GrCmulti

Location of the electrode for the current injections

In [ ]:
soma_loc = ephys.locations.NrnSeclistCompLocation(
        name='soma',
        seclist_name='somatic',
        sec_index=0,
        comp_x=0.5)

Stimuli, recordings location and protocols. For each protocol there is a recording location and a stimulus, in this case are both placed on the the soma.

The delay, duration and total duration can be changed but, more simulation time requires more computational time.

In [ ]:
sweep_protocols = []
for protocol_name, amplitude in [('step1', 0.01), ('step2', 0.016), ('step3', 0.022)]:#
    stim = ephys.stimuli.NrnSquarePulse(
                step_amplitude=amplitude,
                step_delay=100,
                step_duration=2000,
                location=soma_loc,
                total_duration=2200)
    rec = ephys.recordings.CompRecording(
            name='%s.soma.v' % protocol_name,
            location=soma_loc,
            variable='v')
    protocol = ephys.protocols.SweepProtocol(protocol_name, [stim], [rec])
    sweep_protocols.append(protocol)
threestep_protocol = ephys.protocols.SequenceProtocol('twostep', protocols=sweep_protocols)

Loading of NEURON as the simulator

In [ ]:
nrn = ephys.simulators.NrnSimulator()

These conductances are based on the original non optimized model to give an idea of how the optimized cells have to responde to be declared validated.

In [ ]:
default_params = {'gcabar_GRC_CA_dend':0.012771,
		  'gkbar_GRC_KCA_dend':0.003411,
		  'gl_GRC_LKG1_dend': 0.000025,
		  'ggaba_GRC_LKG2_dend': 0.000049,
		  'beta_GRC_CALC_dend': 0.609785, 
		  'cai0_GRC_CALC_dend': 0.0025,
		  'gkbar_GRC_KIR_soma': 0.002689,
		  'gkbar_GRC_KA_soma':0.009488,
		  'gl_GRC_LKG1_soma':0.000127,
		  'gkbar_GRC_KM_soma':0.000209,
		  'beta_GRC_CALC_soma':1.496075,
		  'cai0_GRC_CALC_soma':0.000120,
		  'gnabar_GRC_NA_axon0': 0.386557,
		  'gkbar_GRC_KV_axon0':0.027823,
		  'gl_GRC_LKG1_axon0':0.000120,
		  'gnabar_GRC_NA_axon1':0.002765,
		  'gkbar_GRC_KV_axon1':0.003350,
		  'gl_GRC_LKG1_axon1':0.000010}
		  
responses = threestep_protocol.run(cell_model=GrCmulti, param_values=default_params, sim=nrn)

Plotting the response traces:

3 traces are plotted in a graph, each trace generated by a different current injection step.

In [ ]:
def plot_responses(responses):
    plt.subplot(3,1,1)
    plt.plot(responses['step1.soma.v']['time'], responses['step1.soma.v']['voltage'], label='step1')
    plt.legend()
    plt.subplot(3,1,2)
    plt.plot(responses['step2.soma.v']['time'], responses['step2.soma.v']['voltage'], label='step2')
    plt.legend()
    plt.subplot(3,1,3)
    plt.plot(responses['step3.soma.v']['time'], responses['step3.soma.v']['voltage'], label='step3')
    plt.legend()
    plt.tight_layout()

plot_responses(responses)
plt.show()

List of all the features to be used for each step of current injection.
All these information were taken from in vitro traces obtain from experiments on mice.

In [ ]:
efel_feature_means = {'step1': {'AP_height': 20.93,
                    'ISI_CV':0.261,
                    'AHP_depth_abs_slow':-52.69,
                    'AP_width':0.665,
                    'voltage_base':-68.5,
                    'AHP_depth_abs':-59.21,
                    'time_to_first_spike':31.9,
                    'adaptation_index2':0.1062,
                    'mean_frequency': 25}, 

                    'step2': {'AP_height': 19.255,
                    'ISI_CV':0.14,
                    'AHP_depth_abs_slow':-48.935,
                    'AP_width':0.695,
                    'voltage_base':-68.77,
                    'AHP_depth_abs':-58.3,
                    'time_to_first_spike':19.0,
                    'adaptation_index2':0.034,
                    'mean_frequency': 40}, 
                  
                    'step3': {'AP_height': 17.645,
                    'ISI_CV':0.148,
                    'AHP_depth_abs_slow':-32.67,
                    'AP_width':0.7135,
                    'voltage_base':-69.125,
                    'AHP_depth_abs':-57.191,
                    'time_to_first_spike':14.65,
                    'adaptation_index2':0.029,
                    'mean_frequency': 50}}

Objectives. The function that try to match what was defined above, for example the speed, with the actual result in the individual obtained during each generation.

As per BluePyOpt default, each feature will be an objective.

In [ ]:
objectives = []

for protocol in sweep_protocols:
    stim_start = protocol.stimuli[0].step_delay
    stim_end = stim_start + protocol.stimuli[0].step_duration
    for efel_feature_name, mean in efel_feature_means[protocol.name].iteritems():
        feature_name = '%s.%s' % (protocol.name, efel_feature_name)
        feature = ephys.efeatures.eFELFeature(
                    feature_name,
                    efel_feature_name=efel_feature_name,
                    recording_names={'': '%s.soma.v' % protocol.name},
                    stim_start=stim_start,
                    stim_end=stim_end,
                    exp_mean=mean,
                    exp_std=0.1 * mean)
	objective = ephys.objectives.SingletonObjective(feature_name, feature)
	objectives.append(objective)
        

Objective evaluator. Which calculates the quality of each results.

In [ ]:
score_calc = ephys.objectivescalculators.ObjectivesCalculator(objectives) 

Cell Evaluator. The conductances that will be evaluated during the optimization

In [ ]:
cell_evaluator = ephys.evaluators.CellEvaluator(
        cell_model=GrCmulti,
        param_names=['gcabar_GRC_CA_dend', 
              'gkbar_GRC_KCA_dend',     
              'gl_GRC_LKG1_dend',
              'ggaba_GRC_LKG2_dend',
              'beta_GRC_CALC_dend', 
              'cai0_GRC_CALC_dend',
              'gkbar_GRC_KIR_soma',
              'gkbar_GRC_KA_soma',
              'gl_GRC_LKG1_soma',
              'gkbar_GRC_KM_soma',
              'beta_GRC_CALC_soma',
              'cai0_GRC_CALC_soma',
              'gnabar_GRC_NA_axon0',
              'gkbar_GRC_KV_axon0',
              'gl_GRC_LKG1_axon0',
              'gnabar_GRC_NA_axon1',
              'gkbar_GRC_KV_axon1',
              'gl_GRC_LKG1_axon1'],
    
        fitness_protocols={threestep_protocol.name: threestep_protocol},
        fitness_calculator=score_calc,
        sim=nrn)

print cell_evaluator.evaluate_with_dicts(default_params)

Optimization. Number of offsprings and max generation

In [ ]:
optimisation = bpop.optimisations.DEAPOptimisation(
        evaluator=cell_evaluator,
        offspring_size = NUMBER_INDIVIDUALS,
        seed=os.getenv('BLUEPYOPT_SEED'))


final_pop, hall_of_fame, logs, hist = optimisation.run(max_ngen=NUMBER_GENERATIONS)

Evaluation of the best individual based on its simulations, which are performed with the code below.

In [ ]:
#best individual
best_ind = hall_of_fame[0]
print 'Best individual: ', best_ind
print 'Fitness values: ', best_ind.fitness.values

best_ind_dict = cell_evaluator.param_dict(best_ind)
print cell_evaluator.evaluate_with_dicts(best_ind_dict)

plot_responses(threestep_protocol.run(cell_model=GrCmulti, param_values=best_ind_dict, sim=nrn))

Plot of the best individual and the min fitness obtained during the optimization 

In [ ]:
#import numpy
gen_numbers = logs.select('gen')
min_fitness = logs.select('min')
max_fitness = logs.select('max')
plt.plot(gen_numbers, min_fitness, label='min fitness')
plt.xlabel('generation #')
plt.ylabel('score (# std)')
plt.legend()
plt.xlim(min(gen_numbers) - 1, max(gen_numbers) + 1) 
plt.ylim(0.9*min(min_fitness), 1.1 * max(min_fitness)) 
plt.show()